# Analysis of Car Listings on eBay Kleinanzeigen 


## Description
A dataset with 50,000 datapoints will be used that has been scraped from the German eBay section's website. The aim of this project is to analyze the included used car listings after the dataset has been cleaned. 

## Summary
The data dictionary provided with data is as follows:

- `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
- `name` - Name of the car.
- `seller` - Whether the seller is private or a dealer.
- `offerType` - The type of listing
- `price` - The price on the ad to sell the car.
- `abtest` - Whether the listing is included in an A/B test.
- `vehicleType` - The vehicle Type.
- `yearOfRegistration` - The year in which the car was first registered.
- `gearbox` - The transmission type.
- `powerPS` - The power of the car in PS.
- `model` - The car model name.
- `kilometer` - How many kilometers the car has driven.
- `monthOfRegistration` - The month in which the car was first registered.
- `fuelType` - What type of fuel the car uses.
- `brand` - The brand of the car.
- `notRepairedDamage` - If the car has a damage which is not yet repaired.
- `dateCreated` - The date on which the eBay listing was created.
- `nrOfPictures` - The number of pictures in the ad.
- `postalCode` - The postal code for the location of the vehicle.
- `lastSeenOnline` - When the crawler saw this ad last online.

The dataset is in German.

In [ ]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding='Latin-1')

autos

In [ ]:
autos.info()
autos.head()

# Observations on the Data

The dataset consists of a total of 20 columns. It contains five integer dtypes and 15 string objects. The columns `vehicleType`, `gearbox`, `model`, `fuelType` and `notrepairedDamage` contain NaN, but none have more than ~20% null values. The column names are in camelcase.

In [ ]:
autos.columns

In [ ]:
#list of column names in snakecase
snake_cols=['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

#assign edited column names
autos.columns=snake_cols

autos.head()

In [ ]:
#get descriptive stats on the whole dataset
autos.describe(include='all')

# Descriptive Statistics for all Columns

Text columns where all or almost all values are the same can often be dropped as they don't have useful information for analysis. 
The descriptive statistics table above reveals that:
- Out of 50,000 observations in the `seller` column, 49,999 are `privat`
- Out of 50,000 observations in the `offer_type` column, 49,999 are `Angebot`
- All values in the `nr_of_pictures` columns appear to be zero 
- Most offers don't seem to have included a postal code in the `postal_code` column
- Values in the `price` and in the `odometer` column are stored as text

In [ ]:
#all privat, except one gewerblich
print(autos['seller'].value_counts())
#all Angebot, except one Gesuch
print(autos['offer_type'].value_counts())
#all 0s
print(autos['nr_of_pictures'].value_counts())


In [ ]:
#Remove non-numeric characters and convert to numeric dtype for price and odometer
autos["price"]=(autos["price"].str.replace("$", "")
                .str.replace(",", "").astype(int)
               )

autos["odometer"]=(autos["odometer"].str.replace("km", "")
                   .str.replace(",","")
                  .astype(int)
                  )

#rename col odometer to odometer_km
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

#drop seller, offer_type and nr_of_pictures cols
autos.drop(['seller', 'offer_type', 'nr_of_pictures'], axis=1, inplace=True)

In [ ]:
#check if numeric conversion and re-labeling worked
print(autos.info())

In [ ]:
#function to explore data further
def explore_data(series):
    unique_vals=series.unique().shape[0]
    print("{s} has {u} unique values".format(s=series.name, 
                                             u=unique_vals))
    print('Descriptive Stats')
    print(series.describe().round(2))
    print('Variations of Value Counts')
    print(series.value_counts().
          sort_index(ascending=True)
         )
          
#call explore data on price and odometer_km columns
explore_data(autos["price"])
print('\n')
explore_data(autos["odometer_km"])

In [ ]:
#exclude outliers from price col from the dataset
autos = autos[autos["price"].between(1,999999)]

# Outliers in the Data

## Price
The maximum possible price to enter seems to be 999,999, prices above seem to have been used as placeholder prices. But even if these were legit prices, there are only very few observations above this point which would bias the sample.
The minimum price is 0, which is counted 1,421 times. 
Therefore, all observations that are equal to 0 and above 999,999 will be removed. 

## Odometer_km
There are no abnormal observations in this column, so no datapoints will be removed. 

In [ ]:
def explore_date(series):
    #include missing values in the distribution and use percentages instead of counts
    print(series.str[:10].value_counts(normalize=True, dropna=False)
          .sort_index(ascending=True)
         )
    
print('Date crawled')
explore_date(autos["date_crawled"])
print('\n')
print('Ad created')
explore_date(autos["ad_created"])
print('\n')
print('Last seen')
explore_date(autos["last_seen"])

In [ ]:
print(autos["registration_year"].describe())

## Date columns
The distribution of `date_crawled` shows that the dataset has been crawled between May and July 2016. There seem to be no noticeable accumulations in the `ad_created` and `last_seen` columns. 
In the column `registration_year`, we do find some outliers, namely year 1000 as minimum and 9999 as maximum, both need to be excluded from the dataset. As the data was crawled in 2016, excluding datapoints with registration years earlier than 1900 and later than 2016 makes sense. 

In [ ]:
autos=autos[autos["registration_year"].between(1900,2016)]

print('Value Count')
autos["registration_year"].value_counts(normalize=True).sort_index(ascending=True)

The values in the column registration_year seem to be plausible now, with a lower bound of 1910 and upper bound of 2016. There are again no extreme accumulations, but most cars offered seem to be from the late 1990s to late 2000s

In [ ]:
#find the 20 most common brands
brand_count = autos["brand"].value_counts().head(21)
print(brand_count)

## Brands
There are in total over 40 brands in the dataset, which is why it makes sense to aggregate. Therefore, I will perform further analysis only on the 20 most common brands.

In [ ]:
mean_price_brand={}
top_20=brand_count[brand_count>408].index

for t in top_20:
    sel_rows=autos[autos["brand"]==t]
    mean_price=sel_rows["price"].mean().round(2)
    mean_price_brand[t]=float(mean_price)
    
print(mean_price_brand)

The most popular brand Volkswagen is being offered at a mean price of 5,604.07, while the least popular out of the top 20 popular brands, Mini, is being offered at a mean price of 10,613.46. In general, Mini, Audi, BMW and Mercedes Benz are more expensive and Fiat, Renault and Opel are less expensive with mean prices below 3,000.

In [ ]:
mean_mileage_brand={}
#top_6=brand_count[brand_count>3263].index

for t in top_20:
    sel_rows=autos[autos["brand"]==t]
    #mean_price=sel_rows["price"].mean().round(2)
    mean_mileage=sel_rows["odometer_km"].mean().round(2)
    mean_mileage_brand[t]=float(mean_mileage)
    
#Convert both dictionaries to series objects
bmp_series = pd.Series(mean_price_brand)
bmm_series = pd.Series(mean_mileage_brand)

#Create a dataframe from bmp_series
bmp_df = pd.DataFrame(bmp_series, columns=["mean_price"])

#Assign the bmm_series as a new column
bmp_df["mean_mileage"]=bmm_series

bmp_df.sort_values("mean_price", ascending=True)

# Conclusion
The reason why Mini is the most expensive brand seems to be the relatively low average mileage, which is at ca. 88,000 km. "Sonstige Autos" (aggregated other brands) shows an even higher mean price with a comparable mean mileage. Hence, there seems to be an inverse correlation between mileage and price of used cars. Nevertheless, this doesn't seem to be a very consistent relationship; f.e. Audi cars are among the most expensive brands, but show almost the same mean mileage than the much cheaper Ford cars (129157.39 km vs. 124266.24
